In [2]:
import numpy as np
import scipy as sp

In [3]:
def rodrigez(fi,p):
    e=np.eye(3)
    px=np.array([[0,-p[2],p[1]],
                 [p[2],0,-p[0]],
                 [-p[1],p[0],0]])
    ppT=np.array([[p[0]**2,p[0]*p[1],p[0]*p[2]]
                 ,[p[1]*p[0],p[1]**2,p[1]*p[2]]
                 ,[p[2]*p[0],p[2]*p[1],p[2]**2]])
    return (1-np.cos(fi))*ppT+np.cos(fi)*e+np.sin(fi)*px

In [27]:
def nadji_vektor(sopstveni_vektori):
    for i in range(len(sopstveni_vektori)):
        j=0;
        for elem in sopstveni_vektori[:,i]:
            if elem.imag != 0:
                break
            j+=1;
        if j==3:
            return sopstveni_vektori[:,i]
    raise Exception("Nema odgovarajuceg vektora!")

In [32]:
def A2AxisAngle(A):
# 1. Odredi jedinicni sopstveni vektor p za lambda=1
# 2. Odredi proizvoljan jedinicni vektor u normalan na p
# 3. Odredi u_prim koji je jednak A*u (u_prim je jedinicni)
# 4. Odredi fi koji je jednak arccos(u,u_prim)
# 5. Ako je mesoviti proizvod u,u_prim,p < 0, onda p=-p
# 1.
    sopstvene_vrednosti,sopstveni_vektori=sp.linalg.eig(A,left=True,right=False)
    # p=sopstveni_vektori[:,2] # Problem: Rucno biram odgovarajuci sopstveni vektor, jer nije tacno resenje sa svakim!
    p=nadji_vektor(sopstveni_vektori)
    p=np.array(p)
    p=np.real(p)
# 2. Gram Šmit ortogonalizacija
    u = np.array([1.0,2.0,3.0])
    u -= u.dot(p) * p / np.linalg.norm(p)**2 
    u /= np.linalg.norm(u)
# 3.
    u_prim=A@u
    u_prim/=np.linalg.norm(u_prim)
# 4.    
    fi=np.arccos(u.dot(u_prim)/(np.linalg.norm(u)*np.linalg.norm(u_prim)))
# 5.
    if np.dot(u,np.cross(u_prim,p)) < 0:
        p=-p
    return p,fi

In [13]:
def A2Euler(A):
    if A[2][0] < 1:
        if A[2][0] > -1: #jedinstveno resenje
            psi=np.arctan2(A[1][0],A[0][0])
            teta=np.arcsin(-A[2][0])
            fi=np.arctan2(A[2][1],A[2][2])
            return psi,teta,fi
        else: # Ox3=-Oz
            psi=np.arctan2(-A[0][1],A[1][1])
            teta=np.pi/2
            fi=0
            return psi,teta,fi
    else: #Ox3=Oz
        psi=np.arctan2(-A[0][1],A[1][1])
        teta=-np.pi/2
        fi=0
        return [psi,teta,fi]

In [12]:
def Euler2A(uglovi):
    return rodrigez(uglovi[0],[0,0,1])@rodrigez(uglovi[1],[0,1,0])@rodrigez(uglovi[2],[1,0,0])

In [19]:
def AxisAngle2A(pphi):
    return rodrigez(pphi[3],[pphi[0],pphi[1],pphi[2]])

In [31]:
A=np.array([[1,-8,-4],[4,4,-7],[8,-1,4]],dtype='float32')
A=A/9
print(A)
p,fi=A2AxisAngle(A)
print(fi)
print(p)
rodrigez(fi,p)

[[ 0.11111111 -0.8888889  -0.44444445]
 [ 0.44444445  0.44444445 -0.7777778 ]
 [ 0.8888889  -0.11111111  0.44444445]]
[0.18490002 0.73960031 0.64715018]
[-0.92450036 -0.09245     0.36980006]
1.5707963303609802
[ 0.33333343 -0.6666666   0.6666667 ]


array([[ 0.11111117, -0.88888896, -0.44444433],
       [ 0.44444442,  0.44444439, -0.77777785],
       [ 0.88888893, -0.11111099,  0.44444447]])

In [217]:
A2Euler(A)

(1.3258177, -1.0949141, -0.24497867)

In [14]:
Euler2A(A2Euler(A))

array([[ 0.1111111 , -0.88888886, -0.44444441],
       [ 0.44444444,  0.4444444 , -0.77777776],
       [ 0.8888889 , -0.11111111,  0.44444444]])

In [20]:
AxisAngle2A(np.array([1/3,-2/3,2/3,np.pi/2]))

array([[ 0.11111111, -0.88888889, -0.44444444],
       [ 0.44444444,  0.44444444, -0.77777778],
       [ 0.88888889, -0.11111111,  0.44444444]])

In [22]:
sopstvene_vrednosti,sopstveni_vektori=sp.linalg.eig(A,left=True,right=False)
print(sopstveni_vektori)
B=np.where(sopstveni_vektori.imag == 0, sopstveni_vektori, 0)
print(B)

[[ 0.6666667 +0.j         0.6666667 -0.j         0.33333343+0.j       ]
 [ 0.16666652-0.4999999j  0.16666652+0.4999999j -0.6666666 +0.j       ]
 [-0.16666675-0.5j       -0.16666675+0.5j        0.6666667 +0.j       ]]
[[ 0.6666667 +0.j  0.6666667 -0.j  0.33333343+0.j]
 [ 0.        +0.j  0.        +0.j -0.6666666 +0.j]
 [ 0.        +0.j  0.        +0.j  0.6666667 +0.j]]


In [26]:
print(nadji_vektor(sopstveni_vektori))

[ 0.33333343+0.j -0.6666666 +0.j  0.6666667 +0.j]
